#### Import block

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant

In [2]:
# Check for available GPU.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

#### Function definitions

In [3]:
# Loader functions
# Inspiration: https://towardsdatascience.com/writing-custom-keras-generators-fe815d992c5a

def get_input(path):
    # Load array.
    t_input = np.load(path)
    # Pad to even number of pixels
    t_input = np.pad(t_input, [(0,0),(0,1),(0,1)])
    # Resize to include a channel dimension.
    t_input = tf.expand_dims(t_input, axis = -1)
    return t_input

def get_output(path):
    # Load array.
    t_output = np.load(path)
    # Pad to even number of pixels
    t_output = np.pad(t_output, [(0,0),(0,1),(0,1)])
    # Resize to include a channel dimension.
    t_output = tf.expand_dims(t_output, axis = -1)
    return t_output

def data_generator(samples, batch_size = 64):
    
    while True:
        # Select files (paths/indices) for the batch
        batch_samples  = np.random.choice(a = samples.index, 
                                      size = batch_size)
        batch_input  = []
        batch_output = [] 

        # Read in each input, perform preprocessing and get labels
        for sample in batch_samples:
          input = get_input(samples.loc[sample].features)
          output = get_output(samples.loc[sample].labels)

          batch_input += [input]
          batch_output += [output]
        # Return a tuple of (input, output) to feed the network
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)
        
        yield(batch_x, batch_y)


In [4]:
m = pd.read_csv('Sample_Dataset/train/meta.csv')
m

,Unnamed: 0,Lon,Lat,Date,features,labels
0,356962,-53.345535,-6.535028,2017-07-30,Sample_Dataset/train/features/356962.npy,Sample_Dataset/train/labels/356962.npy
1,546517,-47.329685,-8.260676,2017-08-20,Sample_Dataset/train/features/546517.npy,Sample_Dataset/train/labels/546517.npy
2,799359,-50.967861,-8.255809,2017-09-04,Sample_Dataset/train/features/799359.npy,Sample_Dataset/train/labels/799359.npy
3,714590,-66.144379,-12.624272,2017-08-31,Sample_Dataset/train/features/714590.npy,Sample_Dataset/train/labels/714590.npy
4,240012,-48.350338,-11.838207,2017-07-14,Sample_Dataset/train/features/240012.npy,Sample_Dataset/train/labels/240012.npy
...,...,...,...,...,...,...
1294,436681,-65.542587,-13.089134,2017-08-08,Sample_Dataset/train/features/436681.npy,Sample_Dataset/train/labels/436681.npy
1295,2086150,-61.070286,-23.014650,2017-12-13,Sample_Dataset/train/features/2086150.npy,Sample_Dataset/train/labels/2086150.npy
1296,1733368,-40.953232,-6.312128,2017-10-22,Sample_Dataset/train/features/1733368.npy,Sample_Dataset/train/labels/1733368.npy
1297,1359573,-49.811127,-8.767807,2017-09-25,Sample_Dataset/train/features/1359573.npy,Sample_Dataset/train/labels/1359573.npy


#### Load data

In [5]:
# Get data from one chip
sample_input = np.load('Sample_Dataset/train/features/3243.npy')

# Pad to even number of pixels
a = np.pad(sample_input, [(0,0),(0,1),(0,1)])
# Resize to include a channel dimension.
a = tf.expand_dims(a, axis = -1)
# Resize to include a batch dimension.
a = tf.expand_dims(a, axis = 0)
# Display shape for verification.
a.shape

TensorShape([1, 16, 256, 256, 1])

#### Experiment with basic model structure

In [6]:
b = tf.keras.layers.Conv3D(32, (3,3,3), padding = 'same', activation='relu', bias_initializer=Constant(0.01), 
                           input_shape=(a))(a)
b.shape

TensorShape([1, 16, 256, 256, 32])

In [7]:
c = tf.keras.layers.Conv3D(32, (3,3,3), padding = 'same', activation='relu', bias_initializer=Constant(0.01) 
                           )(b)
c.shape

TensorShape([1, 16, 256, 256, 32])

In [8]:
d = layers.MaxPooling3D((2,4,4))(c)
d.shape

TensorShape([1, 8, 64, 64, 32])

In [9]:
e = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu')(d)
e.shape

TensorShape([1, 8, 64, 64, 64])

In [10]:
f = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu')(e)
f.shape

TensorShape([1, 8, 64, 64, 64])

In [11]:
g = layers.MaxPooling3D((2,2,2))(f)
g.shape

TensorShape([1, 4, 32, 32, 64])

In [12]:
h = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu')(g)
h.shape

TensorShape([1, 4, 32, 32, 128])

In [13]:
i = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu')(h)
i.shape

TensorShape([1, 4, 32, 32, 128])

In [14]:
j = layers.MaxPooling3D((2,2,2))(i)
j.shape

TensorShape([1, 2, 16, 16, 128])

In [15]:
k = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(j)
k.shape

TensorShape([1, 2, 16, 16, 256])

In [16]:
l = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(k)
l.shape

TensorShape([1, 2, 16, 16, 256])

In [17]:
m = layers.MaxPooling3D((1,4,4))(l)
m.shape

TensorShape([1, 2, 4, 4, 256])

In [18]:
n = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu')(m)
n.shape

TensorShape([1, 2, 4, 4, 512])

In [19]:
o = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu')(n)
o.shape

TensorShape([1, 2, 4, 4, 512])

In [20]:
p = layers.MaxPooling3D((1,2,2))(o)
p.shape

TensorShape([1, 2, 2, 2, 512])

In [21]:
q = tf.keras.layers.Conv3D(1024, (2,2,2), padding = 'same', activation='relu')(p)
q.shape

TensorShape([1, 2, 2, 2, 1024])

In [22]:
r = tf.keras.layers.Conv3D(1024, (2,2,2), padding = 'same', activation='relu')(q)
r.shape

TensorShape([1, 2, 2, 2, 1024])

In [23]:
s = layers.MaxPooling3D((2,2,2))(r)
s.shape

TensorShape([1, 1, 1, 1, 1024])

In [24]:
t = layers.Conv3DTranspose(512, (2,2,2), (2,2,2), padding = 'same', activation='relu')(s)
t = layers.concatenate([t, p])
t.shape

TensorShape([1, 2, 2, 2, 1024])

In [25]:
u = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu')(t)
u.shape

TensorShape([1, 2, 2, 2, 512])

In [26]:
v = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu')(u)
v.shape

TensorShape([1, 2, 2, 2, 512])

In [27]:
w = layers.Conv3DTranspose(512, (2,2,2), (1,2,2), padding = 'same', activation='relu')(v)
w = layers.concatenate([w, o])
w.shape

TensorShape([1, 2, 4, 4, 1024])

In [28]:
x = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(w)
x.shape

TensorShape([1, 2, 4, 4, 256])

In [29]:
y = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(x)
y.shape

TensorShape([1, 2, 4, 4, 256])

In [30]:
z = layers.Conv3DTranspose(256, (2,2,2), (1,4,4), padding = 'same', activation='relu')(y)
z = layers.concatenate([z, l])
z.shape

TensorShape([1, 2, 16, 16, 512])

In [31]:
aa = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(z)
aa.shape

TensorShape([1, 2, 16, 16, 256])

In [32]:
ab = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu')(aa)
ab.shape

TensorShape([1, 2, 16, 16, 256])

In [33]:
ac = layers.Conv3DTranspose(128, (2,2,2), (2,2,2), padding = 'same', activation='relu')(ab)
ac = layers.concatenate([ac, i])
ac.shape

TensorShape([1, 4, 32, 32, 256])

In [34]:
ad = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu')(ac)
ad.shape

TensorShape([1, 4, 32, 32, 128])

In [35]:
ae = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu')(ad)
ae.shape

TensorShape([1, 4, 32, 32, 128])

In [36]:
af = layers.Conv3DTranspose(64, (2,2,2), (2,2,2), padding = 'same', activation='relu')(ae)
af = layers.concatenate([af, f])
af.shape

TensorShape([1, 8, 64, 64, 128])

In [37]:
ag = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu')(af)
ag.shape

TensorShape([1, 8, 64, 64, 64])

In [38]:
ah = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu')(ag)
ah.shape

TensorShape([1, 8, 64, 64, 64])

In [39]:
ai = layers.Conv3DTranspose(32, (2,2,2), (1,4,4), padding = 'same', activation='relu')(ah)
ag = layers.MaxPooling3D((2,1,1))(c)
ai = layers.concatenate([ai, ag])
ai.shape

TensorShape([1, 8, 256, 256, 64])

In [40]:
ak = layers.Conv2D(1, 1, padding='same', activation = 'sigmoid')(ai)
ak.shape

TensorShape([1, 8, 256, 256, 1])

#### Model assembly

In [41]:
inputs = layers.Input(shape=((16,256,256,1)))
b = tf.keras.layers.Conv3D(32, (3,7,7), padding = 'same', activation='relu', bias_initializer=Constant(0.01), 
                           )(inputs)
c = tf.keras.layers.Conv3D(32, (3,5,5), padding = 'same', activation='relu', bias_initializer=Constant(0.01) 
                           )(b)
d = layers.MaxPooling3D((2,4,4))(c)
d = layers.Dropout(0.3)(d)
e = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(d)
f = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(e)
g = layers.MaxPooling3D((2,2,2))(f)
g = layers.Dropout(0.3)(g)
h = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(g)
i = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(h)
j = layers.MaxPooling3D((2,2,2))(i)
j = layers.Dropout(0.3)(j)
k = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(j)
l = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(k)
m = layers.MaxPooling3D((1,4,4))(l)
m = layers.Dropout(0.3)(m)
n = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(m)
o = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(n)
p = layers.MaxPooling3D((1,2,2))(o)
p = layers.Dropout(0.3)(p)
q = tf.keras.layers.Conv3D(1024, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(p)
r = tf.keras.layers.Conv3D(1024, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(q)
s = layers.MaxPooling3D((2,2,2))(r)
s = layers.Dropout(0.3)(s)
t = layers.Conv3DTranspose(512, (2,2,2), (2,2,2), padding = 'same', activation='relu')(s)
t = layers.concatenate([t, p])
t = layers.Dropout(0.3)(t)
u = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(t)
v = tf.keras.layers.Conv3D(512, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(u)
w = layers.Conv3DTranspose(512, (2,2,2), (1,2,2), padding = 'same', activation='relu')(v)
w = layers.concatenate([w, o])
w = layers.Dropout(0.3)(w)
x = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(w)
y = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(x)
z = layers.Conv3DTranspose(256, (2,2,2), (1,4,4), padding = 'same', activation='relu')(y)
z = layers.concatenate([z, l])
z = layers.Dropout(0.3)(z)
aa = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(z)
ab = tf.keras.layers.Conv3D(256, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(aa)
ac = layers.Conv3DTranspose(128, (2,2,2), (2,2,2), padding = 'same', activation='relu')(ab)
ac = layers.concatenate([ac, i])
ac = layers.Dropout(0.3)(ac)
ad = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(ac)
ae = tf.keras.layers.Conv3D(128, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(ad)
af = layers.Conv3DTranspose(64, (2,2,2), (2,2,2), padding = 'same', activation='relu')(ae)
af = layers.concatenate([af, f])
af = layers.Dropout(0.3)(af)
ag = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(af)
ah = tf.keras.layers.Conv3D(64, (2,2,2), padding = 'same', activation='relu', bias_initializer=Constant(0.01))(ag)
ai = layers.Conv3DTranspose(32, (2,2,2), (1,4,4), padding = 'same', activation='relu')(ah)
ag = layers.MaxPooling3D((2,1,1))(c)
ai = layers.concatenate([ai, ag])
outputs = layers.Conv2D(1, 1, padding='same', activation = 'sigmoid')(ai)

In [42]:
forecast_model = tf.keras.Model(inputs, outputs, name="3D_UNet")

In [43]:
# Verify output shape.
forecast_model.predict(a).shape

1/1 [==============================] - 0s 314ms/step


(1, 8, 256, 256, 1)

In [44]:
# Display model details.
forecast_model.summary()

Model: "3D_UNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16, 256, 25  0           []                               
                                6, 1)]                                                            
                                                                                                  
 conv3d_22 (Conv3D)             (None, 16, 256, 256  4736        ['input_1[0][0]']                
                                , 32)                                                             
                                                                                                  
 conv3d_23 (Conv3D)             (None, 16, 256, 256  76832       ['conv3d_22[0][0]']              
                                , 32)                                                       

                                                                                                  
 dropout_6 (Dropout)            (None, 2, 2, 2, 102  0           ['concatenate_6[0][0]']          
                                4)                                                                
                                                                                                  
 conv3d_34 (Conv3D)             (None, 2, 2, 2, 512  4194816     ['dropout_6[0][0]']              
                                )                                                                 
                                                                                                  
 conv3d_35 (Conv3D)             (None, 2, 2, 2, 512  2097664     ['conv3d_34[0][0]']              
                                )                                                                 
                                                                                                  
 conv3d_tr

Total params: 34,987,457
Trainable params: 34,987,457
Non-trainable params: 0
__________________________________________________________________________________________________


In [46]:
# Compile model.
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
forecast_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [47]:
# Data loaders.
batch_size = 2
meta_t = pd.read_csv('Sample_Dataset/train/meta.csv')
meta_v = pd.read_csv('Sample_Dataset/val/meta.csv')
t_gen = data_generator(meta_t, batch_size = batch_size)
v_gen = data_generator(meta_v, batch_size = batch_size)

In [48]:
# Train model.
forecast_model.fit(t_gen, 
                   epochs = 5, 
                   verbose = 1, 
                   validation_data = v_gen, 
                   steps_per_epoch = len(meta_t) // batch_size,
                   validation_steps = len(meta_v) // batch_size,
                  )

Epoch 1/5
649/649 [==============================] - 232s 354ms/step - loss: 0.0858 - accuracy: 0.9983 - val_loss: 0.0044 - val_accuracy: 0.9998
Epoch 2/5
649/649 [==============================] - 226s 349ms/step - loss: 0.0049 - accuracy: 0.9995 - val_loss: 0.0028 - val_accuracy: 0.9998
Epoch 3/5
649/649 [==============================] - 226s 348ms/step - loss: 0.0043 - accuracy: 0.9995 - val_loss: 0.0024 - val_accuracy: 0.9998
Epoch 4/5
649/649 [==============================] - 225s 347ms/step - loss: 0.0042 - accuracy: 0.9996 - val_loss: 0.0022 - val_accuracy: 0.9998
Epoch 5/5
649/649 [==============================] - 225s 347ms/step - loss: 0.0043 - accuracy: 0.9995 - val_loss: 0.0022 - val_accuracy: 0.9998


In [50]:
# Save trained model.
forecast_model.save('Models/001')

INFO:tensorflow:Assets written to: Models/001\assets


INFO:tensorflow:Assets written to: Models/001\assets


#### Examine how model is working

In [51]:
# Get data from one chip
sample_input = np.load('Sample_Dataset/val/features/2128240.npy')

# Pad to even number of pixels
a = np.pad(sample_input, [(0,0),(0,1),(0,1)])
# Resize to include a channel dimension.
a = tf.expand_dims(a, axis = -1)
# Resize to include a batch dimension.
a = tf.expand_dims(a, axis = 0)

# Get data from one chip
sample_output = np.load('Sample_Dataset/val/labels/2128240.npy')

# Pad to even number of pixels
b = np.pad(sample_output, [(0,0),(0,1),(0,1)])
# Resize to include a channel dimension.
b = tf.expand_dims(b, axis = -1)
# Resize to include a batch dimension.
b = tf.expand_dims(b, axis = 0)

In [52]:
pred_eval = forecast_model.predict(a)
pred_eval.shape

1/1 [==============================] - 0s 231ms/step


(1, 8, 256, 256, 1)

In [54]:
pred_eval.sum()

268.0803

In [55]:
b.shape

TensorShape([1, 8, 256, 256, 1])

In [61]:
b.numpy().sum()

38.0

In [71]:
pred_eval[0][0][0]

array([[0.0447113 ],
       [0.01946915],
       [0.00815297],
       [0.00815297],
       [0.00797041],
       [0.00822825],
       [0.00815297],
       [0.00815297],
       [0.00722208],
       [0.00768635],
       [0.00815297],
       [0.00815297],
       [0.00797008],
       [0.00822819],
       [0.00815297],
       [0.00815297],
       [0.00722223],
       [0.00768676],
       [0.00815297],
       [0.00815297],
       [0.00797036],
       [0.00822793],
       [0.00815297],
       [0.00815297],
       [0.00722226],
       [0.00768642],
       [0.00815297],
       [0.00815297],
       [0.0079699 ],
       [0.008228  ],
       [0.00815297],
       [0.00815297],
       [0.00722216],
       [0.00768731],
       [0.00815297],
       [0.00815297],
       [0.00796969],
       [0.00822788],
       [0.00815297],
       [0.00815297],
       [0.00722216],
       [0.00768655],
       [0.00815297],
       [0.00815297],
       [0.00796958],
       [0.00822765],
       [0.00815297],
       [0.008

In [84]:
# Convert predictions to binary.
(pred_eval >= 0.5).astype(int)

array([[[[[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         ...,

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]]],


        [[[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         ...,

     

In [85]:
pred_eval.max()

0.04906586